In [13]:
import os
import chess
import numpy as np
import pandas as pd
import keras


nOfSamples = 100000 #Number of samples to train with

os.chdir('/Users/Nasif/Chess AI')
df = pd.read_csv('chessData.csv')

df=df.drop(df.index[nOfSamples:]) #Load FEN and Evaluation value data into a pandas dataframe

#df.dtypes

FEN           object
Evaluation    object
dtype: object

In [14]:
#One hot encoding of chess pieces dictionary
chess_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0],
} 

def make_matrix(fenString): #Converts FEN data to a ASCII representation of the chess board

    #set up board from FEN data
    board = chess.Board()
    board.set_fen(fenString)
    fen = board.fen()
    
    tempArr1 = [] 
    #After FEN data inputted in board in chess library, extract the piece position information
    pieces = fen.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        tempArr2 = []  
        for num in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    tempArr2.append('.')
            else:
                tempArr2.append(thing)
        tempArr1.append(tempArr2)
    return tempArr1

def translate(matrix,chess_dict): #Converts ASCII representation to matrix for NN input
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(chess_dict[term])
        rows.append(terms)
    return rows

#print(translate(make_matrix('rnbqkb1r/pppppppp/B4n2/8/4P3/8/PPPP1PPP/RNBQK1NR b KQkq - 2 2'),chess_dict))

[[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 

In [15]:
#Process dataset for NN input

def NormalizeData(data): #Normalize evaluation values in the dataset
    normDataSet = []
    minY = (np.min(data))
    maxY = (np.max(data))
    for x in data:
        temp = (((x - minY)/(maxY-minY)))
        normDataSet.append(temp)
    return normDataSet

def data_setup():
    X = []
    y = []
    testX = []
    testY = []
    i = 0
    while (i<nOfSamples-200):
        matrix = make_matrix(df.iloc[i,0])
        trans_matrix = translate(matrix,chess_dict)
        temp = df.iloc[i,1]
        if '#' not in temp:
            y.append(int(temp))
            X.append(trans_matrix)

        i += 1
    while (i<nOfSamples):
        matrix = make_matrix(df.iloc[i,0])
        trans_matrix = translate(matrix,chess_dict)
        temp = df.iloc[i,1]
        if '#' not in temp:
            testY.append(int(temp))
            testX.append(trans_matrix)
        i += 1
    return X,y,testX,testY

X,y,testX,testY = data_setup()
del[df]
#y,minY,maxY = NormalizeData(y)
#print(minY)
#print(maxY)
testY = NormalizeData(testY)

-9024
7685


In [ ]:
#Setup Keras learning network

from keras.models import Sequential
from keras.layers import Dropout, Dense, Activation, Flatten
from keras.optimizers import SGD



def initialize_network(): #Multi Layer Perceptron
    #Instantiate
    model = Sequential()
    #Input layer
    model.add(Flatten(input_shape=(8, 8, 12)))
    #model.add(Activation("elu"))
    
    #Hidden Layers, and dropout layer for batch normalization
    model.add(Dense(2048,input_dim=768, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2048, input_dim=768, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2048, input_dim=768, activation='relu'))
    model.add(Dropout(0.2))
    
    #Output Layer
    model.add(Dense(1))
    #model.add(Activation("softmax"))
    model.add(Activation("tanh"))
    
    #Optimizer for training
    sgd = SGD(learning_rate=0.01, decay=1e-2, momentum=0.7, nesterov=True)
    model.compile(loss='mse',optimizer=sgd,metrics=['mae']) #SGD optimizer used for linear regression
    
    #Train by running the fitting function
    hist = model.fit(X,y,epochs=500,verbose=1) #train for 500 generations
    
    return model #Returns trained network

#Run training model    
model = initialize_network()
results = model.evaluate(testX, testY, batch_size=256)
print("test loss, test acc:", results)
model.save('Models/test2.h5')

Epoch 1/500
1228/3102 [==========>...................] - ETA: 1:53 - loss: 0.0046 - mae: 0.0480

In [ ]:
import math
error = model.evaluate(testX, testY, verbose=0)
print('MSE: %.3f, RMSE: %.3f' % (error, math.sqrt(error)))